# 5-degree monthly average gridded datasets for SIF and XCO2 over Continental US
---

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd 
import xarray as xr

from krige_tools import land_grid
from stat_tools import apply_detrend

In [2]:
# collect gridded dataset, standardize, and establish a common domain
df_grid = (
    pd.DataFrame(
        land_grid(
            res=5, lon_lwr=-125, lon_upr=-65, lat_lwr=10, lat_upr=50
        ))
    .rename(columns={0:"lat", 1:"lon"})
    .assign(land=lambda x:1)
    .set_index(["lon", "lat"])
)

## Detrended, .csv format

In [3]:
# NOTE: no longer standardizing across all time; within year only for now
def detrend_standardize_da(da):
    da, _ = apply_detrend(da)
    return (da - da.mean(dim="time")) / da.std(dim="time")

with xr.open_dataset("../data/exp_pro/OCO2_5deg_monthly.nc") as ds:
    ds = ds[["sif", "xco2"]]
    ds["sif_res"], _ = apply_detrend(ds.sif)
    ds["xco2_res"], _ = apply_detrend(ds.xco2)

df = ds.to_dataframe().join(df_grid).dropna(subset=["land"]).reset_index()
df

,lat,lon,time,sif,xco2,sif_res,xco2_res,land
0,12.5,-92.5,2014-09-01,0.559724,393.029785,0.125447,-3.740133,1.0
1,12.5,-92.5,2014-10-01,0.588696,393.789459,0.154450,-3.207281,1.0
2,12.5,-92.5,2014-11-01,0.393522,396.065002,-0.040693,-1.158559,1.0
3,12.5,-92.5,2014-12-01,0.310282,397.139435,-0.123902,-0.310949,1.0
4,12.5,-92.5,2015-01-01,0.243238,397.303528,-0.190915,-0.373678,1.0
...,...,...,...,...,...,...,...,...
5845,47.5,-67.5,2020-07-01,0.644079,408.272064,0.411713,-4.123634,1.0
5846,47.5,-67.5,2020-08-01,0.463758,406.768005,0.231082,-5.845368,1.0
5847,47.5,-67.5,2020-09-01,0.294015,407.222992,0.061029,-5.608058,1.0
5848,47.5,-67.5,2020-10-01,0.113518,410.960052,-0.119777,-2.088674,1.0


In [4]:
df_conus = df[["lat", "lon", "time", "sif_res", "xco2_res"]]
df_conus.to_csv("../data/exp_pro/OCO2_5deg_monthly_conus_detrended.csv", index=False)

## Raw, .nc format

In [6]:
ds = xr.open_dataset("../data/exp_pro/OCO2_5deg_monthly.nc")
df = ds.to_dataframe().join(df_grid).dropna(subset=["land"]).reset_index()
ds_conus = df.drop(columns=["land"]).set_index(["lon", "lat", "time"]).to_xarray()
ds_conus

<xarray.Dataset>
Dimensions:   (lat: 8, lon: 12, time: 75)
Coordinates:
  * lon       (lon) float64 -122.5 -117.5 -112.5 -107.5 ... -77.5 -72.5 -67.5
  * lat       (lat) float64 12.5 17.5 22.5 27.5 32.5 37.5 42.5 47.5
  * time      (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-11-01
Data variables:
    sif       (lon, lat, time) float32 nan nan nan ... 0.11351814 0.053924084
    sif_var   (lon, lat, time) float32 nan nan nan ... 0.08074269 0.14549519
    xco2      (lon, lat, time) float32 nan nan nan nan ... 407.223 410.96005 nan
    xco2_var  (lon, lat, time) float32 nan nan nan ... 1.0527697 1.2264916 nan

In [7]:
ds_conus.to_netcdf("../data/exp_pro/OCO2_5deg_monthly_conus.nc", format="NETCDF4")
ds_conus.close()